<a href="https://colab.research.google.com/github/ducthang140/x2frame/blob/main/x2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install mediapy
!sudo apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import subprocess
import cv2
import requests
import numpy as np

from typing import Generator, Iterable, List, Optional
import mediapy as media

In [9]:
model = hub.load("https://tfhub.dev/google/film/1")

In [10]:
_UINT8_MAX_F = float(np.iinfo(np.uint8).max)

def load_image(img_url: str):
  """Returns an image with shape [height, width, num_channels], with pixels in [0..1] range, and type np.float32."""

  if (img_url.startswith("https")):
    user_agent = {'User-agent': 'Colab Sample (https://tensorflow.org)'}
    response = requests.get(img_url, headers=user_agent)
    image_data = response.content
  else:
    image_data = tf.io.read_file(img_url)

  image = tf.io.decode_image(image_data, channels=3)
  image_numpy = tf.cast(image, dtype=tf.float32).numpy()
  return image_numpy / _UINT8_MAX_F

In [ ]:
def removeDuplicateFrame(input_file, output_file):
    ffmpeg_cmd= [
        "ffmpeg",
        "-i", input_file,
        "-vf", "mpdecimate",
        "-vsync", "vfr",
        "-y", output_file
    ]

    try:
        subprocess.run(ffmpeg_cmd, check=True)
        print("success")
    except subprocess.CalledProcessError as e:
        print("fail")

removeDuplicateFrame('/content/drive/MyDrive/x2frame/15fps.mp4','/content/drive/MyDrive/x2frame/output.mp4')

success


In [6]:
vid = cv2.VideoCapture("/content/drive/MyDrive/x2frame/output.mp4")

count = 0
IframeCount=0
while(True):

    ret,frame = vid.read()

    if ret:
        name = '/content/drive/MyDrive/x2frame/frame/frame' + str(count) + '.jpg'
        print ('Creating...' + name)
        IframeName = '/content/drive/MyDrive/x2frame/Iframe/frame' + str(IframeCount) + '.jpg'
        cv2.imwrite(name, frame)
        cv2.imwrite(IframeName, frame)
        count += 1
        IframeCount+=2
    else:
        break

Creating.../content/drive/MyDrive/x2frame/frame/frame0.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame1.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame2.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame3.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame4.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame5.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame6.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame7.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame8.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame9.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame10.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame11.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame12.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame13.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame14.jpg
Creating.../content/drive/MyDrive/x2frame/frame/frame15.jpg
Creating.../content/drive/MyDrive/x2frame/frame/fr

In [11]:
midframeCount=1
for firstFrame in range(count-1):
  image_1_url = '/content/drive/MyDrive/x2frame/frame/frame' + str(firstFrame) + '.jpg'
  image_2_url = '/content/drive/MyDrive/x2frame/frame/frame' + str(firstFrame+1) + '.jpg'

  time = np.array([0.5], dtype=np.float32)

  image1 = load_image(image_1_url)
  image2 = load_image(image_2_url)

  input = {
    'time': np.expand_dims(time, axis=0), # adding the batch dimension to the time
    'x0': np.expand_dims(image1, axis=0), # adding the batch dimension to the image
    'x1': np.expand_dims(image2, axis=0)  # adding the batch dimension to the image
    }

  mid_frame = model(input)

  frame = mid_frame['image'][0].numpy()
  media.write_image('/content/drive/MyDrive/x2frame/Iframe/frame' + str(midframeCount),frame,'jpeg')
  midframeCount+=2

In [ ]:
print(count)

146


In [15]:
vidcap = cv2.VideoCapture('/content/drive/MyDrive/x2frame/output.mp4')
fps = vidcap.get(cv2.CAP_PROP_FPS)

print(f"{fps} frames per second")

14.985014985014985 frames per second


In [27]:
import glob
import re

img_array = []
numbers = re.compile(r'(\d+)')
def numericalSort(value):
  parts = numbers.split(value)
  parts[1::2] = map(int, parts[1::2])
  return parts

writeCount=0
for filename in sorted(glob.glob('/content/drive/MyDrive/x2frame/Iframe/*') , key=numericalSort):
  img = cv2.imread(filename)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)
  print(writeCount)
  writeCount+=1

out = cv2.VideoWriter('/content/drive/MyDrive/x2frame/after.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps*2, size)

for i in range(len(img_array)):
  out.write(img_array[i])
out.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27